## MODELE DE REGRESSION

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import joblib
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)

### 📂Chargement des données 

In [3]:
data = pd.read_csv("../data/donnees_dpe_73_clean.csv", delimiter=",")
print("Nombre de lignes :", data.shape[0])
print("Nombre de colonnes :", data.shape[1])
data.head(3)

Nombre de lignes : 117708
Nombre de colonnes : 143


,numero_dpe,date_derniere_modification_dpe,date_visite_diagnostiqueur,date_etablissement_dpe,date_reception_dpe,date_fin_validite_dpe,modele_dpe,version_dpe,methode_application_dpe,etiquette_dpe,etiquette_ges,classe_altitude,zone_climatique,type_batiment,type_installation_chauffage,type_installation_ecs,hauteur_sous_plafond,nombre_appartement,nombre_niveau_logement,surface_habitable_immeuble,surface_habitable_logement,classe_inertie_batiment,adresse_ban,numero_voie_ban,nom_rue_ban,nom_commune_ban,code_postal_ban,code_insee_ban,code_departement_ban,code_region_ban,identifiant_ban,score_ban,statut_geocodage,adresse_brut,nom_commune_brut,code_postal_brut,numero_etage_appartement,complement_adresse_batiment,complement_adresse_logement,indicateur_confort_ete,protection_solaire_exterieure,logement_traversant,presence_brasseur_air,inertie_lourde,isolation_toiture,deperditions_enveloppe,deperditions_ponts_thermiques,deperditions_murs,deperditions_planchers_hauts,deperditions_planchers_bas,deperditions_portes,deperditions_baies_vitrees,deperditions_renouvellement_air,qualite_isolation_enveloppe,qualite_isolation_murs,qualite_isolation_plancher_bas,qualite_isolation_menuiseries,ubat_w_par_m2_k,besoin_chauffage,besoin_ecs,besoin_refroidissement,conso_5_usages_ep,conso_5_usages_par_m2_ep,conso_chauffage_ep,conso_ecs_ep,conso_refroidissement_ep,conso_eclairage_ep,conso_auxiliaires_ep,conso_5_usages_ef,conso_5_usages_par_m2_ef,conso_chauffage_ef,conso_ecs_ef,conso_refroidissement_ef,conso_eclairage_ef,conso_auxiliaires_ef,emission_ges_5_usages,emission_ges_5_usages_par_m2,emission_ges_chauffage,emission_ges_ecs,emission_ges_refroidissement,emission_ges_eclairage,emission_ges_auxiliaires,type_energie_n1,conso_5_usages_ef_energie_n1,conso_chauffage_ef_energie_n1,conso_ecs_ef_energie_n1,cout_total_5_usages_energie_n1,cout_chauffage_energie_n1,cout_ecs_energie_n1,emission_ges_5_usages_energie_n1,emission_ges_chauffage_energie_n1,emission_ges_ecs_energie_n1,type_energie_n2,conso_5_usages_ef_energie_n2,conso_chauffage_ef_energie_n2,conso_ecs_ef_energie_n2,cout_total_5_usages_energie_n2,cout_chauffage_energie_n2,cout_ecs_energie_n2,emission_ges_5_usages_energie_n2,emission_ges_chauffage_energie_n2,emission_ges_ecs_energie_n2,cout_total_5_usages,cout_chauffage,cout_ecs,cout_refroidissement,cout_eclairage,cout_auxiliaires,type_energie_principale_chauffage,type_generateur_chauffage_principal,type_installation_chauffage_n1,type_emetteur_installation_chauffage_n1,configuration_installation_chauffage_n1,description_installation_chauffage_n1,conso_chauffage_installation_chauffage_n1,surface_chauffee_installation_chauffage_n1,type_generateur_n1_installation_n1,type_energie_generateur_n1_installation_n1,usage_generateur_n1_installation_n1,conso_chauffage_generateur_n1_installation_n1,type_energie_principale_ecs,type_generateur_chauffage_principal_ecs,type_installation_ecs_n1,configuration_installation_ecs_n1,description_installation_ecs_n1,conso_ef_installation_ecs_n1,nombre_logements_desservis_par_installation_ecs_n1,surface_habitable_desservie_par_installation_ecs_n1,type_installation_solaire_n1,type_generateur_n1_ecs_n1,type_energie_generateur_n1_ecs_n1,usage_generateur_n1_ecs_n1,description_generateur_n1_ecs_n1,volume_stockage_generateur_n1_ecs_n1,conso_ef_generateur_n1_ecs_n1,ventilation_posterieure_2012,production_electricite_pv_kwhep_par_an,Logement,annee_construction,annee_reception_DPE,periode_construction,longitude,latitude
0,2173N0055368S,2023-11-08,2021-06-30,2021-06-30,2021-07-01,2031-06-29,DPE NEUF logement : RT2012,1.0,dpe issu d'une étude thermique réglementaire R...,A,A,NaN,NaN,maison,NaN,NaN,2.5,1.0,1.0,95.4,95.4,NaN,Rue de la Gare 73100 Grésy-sur-Aix,NaN,271 RUE DE LA GARE,Grésy-sur-Aix,73100,73128,73.0,84.0,73128_0144,0.55,adresse géocodée ban à l'adresse,271 RUE DE LA GARE,Grésy-sur-Aix,73100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,très bonne,très bonne,très bonne,très bonne,0.39,NaN,NaN,NaN,5956.5,62.3,3859

### 🧹 Nettoyage et préparation

In [4]:
target = "conso_5_usages_par_m2_ep"

features_user = [
    "annee_construction",
    "surface_habitable_logement",
    "type_batiment",
    "type_energie_principale_chauffage",
    "classe_inertie_batiment",
    "qualite_isolation_murs",
    "qualite_isolation_menuiseries",
    "classe_altitude",
    "logement_traversant"
]

# Vérification de la présence des colonnes
missing_in_data = [f for f in features_user if f not in data.columns]
print("Variables manquantes dans le dataset :", missing_in_data)

# Suppression des lignes avec cible manquante
missing_target_count = data[target].isna().sum()
print(f"Valeurs manquantes dans la cible : {missing_target_count} / {len(data)}")

if missing_target_count > 0:
    data = data.dropna(subset=[target])
    print(f"✅ Lignes supprimées : {missing_target_count}")

X = data[features_user]
y = data[target]

print(f"✅ Données prêtes pour le split : {X.shape[0]} lignes, {X.shape[1]} variables explicatives.")


Variables manquantes dans le dataset : []
Valeurs manquantes dans la cible : 1 / 117708
✅ Lignes supprimées : 1
✅ Données prêtes pour le split : 117707 lignes, 9 variables explicatives.


### ✂️ Split des données

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print("Train :", X_train.shape, " Test :", X_test.shape)

Train : (94165, 9)  Test : (23542, 9)


###  ⚙️ Préprocessing

In [6]:
numeric_features = X_train.select_dtypes(include=[np.number]).columns
categorical_features = X_train.select_dtypes(exclude=[np.number]).columns

numeric_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer([
    ("num", numeric_transformer, numeric_features),
    ("cat", categorical_transformer, categorical_features)
])

### 🧠 Modèles de régression

In [7]:
models = {
    "Linear Regression": LinearRegression(),
    "Random Forest": RandomForestRegressor(n_estimators=300, max_depth=15, random_state=42),
    "Gradient Boosting": GradientBoostingRegressor(n_estimators=300, learning_rate=0.05, max_depth=4, random_state=42)
}


### 🧪 Entraînement et évaluation

In [8]:
results = {}

for name, model in models.items():
    pipe = Pipeline([
        ("preprocess", preprocessor),
        ("model", model)
    ])
    
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    
    r2 = r2_score(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    
    results[name] = {"R2": r2, "RMSE": rmse, "MAE": mae}
    
    print(f"\n📊 {name}")
    print(f"R²: {r2:.3f} | RMSE: {rmse:.2f} | MAE: {mae:.2f}")

results_df = pd.DataFrame(results).T.sort_values(by="R2", ascending=False)
display(results_df)


📊 Linear Regression
R²: 0.579 | RMSE: 95.98 | MAE: 65.75

📊 Random Forest
R²: 0.720 | RMSE: 78.20 | MAE: 47.15

📊 Gradient Boosting
R²: 0.677 | RMSE: 84.10 | MAE: 54.85


,R2,RMSE,MAE
Random Forest,0.720485,78.196698,47.154819
Gradient Boosting,0.676724,84.095574,54.846155
Linear Regression,0.578885,95.981321,65.754838


### 💾 Sauvegarde du meilleur modèle

In [9]:
# 🔎 Sélection du meilleur modèle
best_model_name = results_df["R2"].idxmax()
best_model = models[best_model_name]

# 🎓 Entraînement final
best_model.fit(preprocessor.transform(X_train), y_train)

# 📦 Sauvegarde allégée (modèle et préprocesseur séparés)
os.makedirs("../models", exist_ok=True)
joblib.dump(preprocessor, "../models/preprocessor_conso.pkl", compress=3)
joblib.dump(best_model, f"../models/model_CONSO_{best_model_name.replace(' ','_')}.pkl", compress=3)

print("\n✅ Sauvegarde terminée avec succès (modèle compressé) !")
print(f"📘 Meilleur modèle : {best_model_name} (R²={results_df.loc[best_model_name, 'R2']:.3f})")
print("💾 Fichiers enregistrés : preprocessor_conso.pkl + model_CONSO.pkl")



✅ Sauvegarde terminée avec succès (modèle compressé) !
📘 Meilleur modèle : Random Forest (R²=0.720)
💾 Fichiers enregistrés : preprocessor_conso.pkl + model_CONSO.pkl
